In [1]:
%pip install -q langchain text_generation

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 아래에서 계속 사용할 LLM을 만들어줍니다. 

import warnings

from langchain_community.llms import HuggingFaceTextGenInference

warnings.filterwarnings("ignore")

# 야놀자에서 만든 EEVE Korean 8B 모델을 사용합니다.
llm = HuggingFaceTextGenInference(
    inference_server_url="http://eeve-korean-instruct-10-8b-tgi.serving.70-220-152-1.sslip.io"
)


In [3]:
# CustomEmbedding (SDS임베딩)을 사용합니다. 
# 로컬에서 사용할 때는 import 해주세요. 

from typing import (List)

import requests
from langchain_core.embeddings import Embeddings


class CustomEmbedding(Embeddings):
    def __init__(self):
        self.embed_url = 'http://sds-embed.serving.70-220-152-1.sslip.io/v1/models/embed:predict'

    def call_embed(self, url, texts):
        data = {
            "instances": texts
        }
        response = requests.post(url=url, json=data)
        result = response.json()
        return result['predictions']

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        주어진 텍스트를 임베딩하여 벡터로 반환 합니다.
        """
        embed_list = self.call_embed(url=self.embed_url, texts=texts)
        return embed_list

    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""

        embed_list = self.call_embed(url=self.embed_url, texts=[text])
        return embed_list[0]

In [4]:
# 경고 메시지 무시
import warnings

warnings.filterwarnings("ignore")

In [18]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = """
You are an expert chef with an expertise in Italian dishes.
Explain the recipe of {concept} in around 5 lines in Korean.
"""
prompt = PromptTemplate(
    input_variables=["concept"],
    template=template
)
chain = LLMChain (llm=llm, prompt=prompt)

result = chain.invoke("아보카도")

# 결과 출력
print(result['text'])


아보카도, 또는 'alligator pear'라고 불리는 이 과일은 이탈리아 요리에 풍부하고 크리미한 맛을 더하는 데 훌륭한 재료입니다. 이 레시피에서는 신선하고 영양가 있는 아보카도 살사 토핑을 만들겠습니다. 이 살사를 브루스케타, 피자, 구운 생선 등 다양한 요리에 얹어 풍미를 높일 수 있습니다.

재료:
- 익은 아보카도 2개, 으깨기
- 토마토 2개, 다져서
- 다진 양파 1/4컵
- 다진 신선한 고수 1/4컵
- 다진 신선한 고춧가루 1/4컵
- 올리브유 2 큰술
- 레몬즙 1 큰술
- 소금과 후춧가루 취향껏

만드는 법:
1. 아보카도를 반으로 자르고 씨를 제거한 뒤, 숟가락을 사용하여 과육을 으깨세요. 큰 그릇에 으깬 아보카도를 담으세요.
2. 토마토를 깨끗이 씻어 씨를 제거하고 다져서 아보카도에 넣고 잘 섞어주세요.
3. 양파, 고수, 고춧가루도 그릇에 넣고 섞어주세요.
4. 올리브유, 레몬즙, 소금, 후춧가루를 넣고 재료들을 잘 섞어주세요.
5. 맛이 어우러지도록 살사를 30분 정도 냉장 보관한 후, 다양한 이탈리아 요리에 토핑으로 사용하거나 그냥 나눠 드세요.

이 간단하고 맛있는 아보카도 살사 토핑은 여러분의 이탈리아 요리에 크리미하고 풍미가 가득한 터치를 더해줄 것입니다. 맛있게 드세요!


In [13]:
#위의 복잡한 LLMChain도 LCEL로 간단하게 바꿀 수 있다.
#이건 실행하지 마세요. (LLMChain을 엮으려면 동일한 체인으로 만들어야 해서)

from langchain_core.runnables import RunnablePassthrough

chain = (
    {"concept": RunnablePassthrough()}
    | PromptTemplate.from_template("You are and expert chef with an experties in Italian dishes. Explain the recipe of {concept} in 5 lines in Korean.")
    | llm
)

print(chain.invoke({"concept" : "아보카도"}))




아보카도 치킨 샐러드 레시피:

1. 아보카도를 반으로 자르고 씨를 제거하세요.
2. 껍질을 벗겨서 작은 조각으로 잘라주세요.
3. 닭 가슴살을 썰어 양념을 하고 그릴이나 팬에 구워주세요.
4. 냉장고에 있던 치즈를 갈아주세요.
5. 샐러드 잎 위에 구운 치킨, 아보카도 조각, 갈은 치즈를 올려 드레싱과 함께 제공하세요.

이 레시피는 풍미있고 영양가 있는 아보카도 치킨 샐러드를 만드는 과정을 설명해줍니다. 이 샐러드는 맛있고 건강하며, 요리 실력을 뽐낼 수 있는 좋은 방법입니다.


In [19]:
second_prompt = PromptTemplate(
    input_variables = ["recipe"],
    template = "Make a list of cooking ingredients of {recipe} by putting a prefix symbol",
)

chain_two = LLMChain (llm=llm, prompt=second_prompt)



In [20]:
chain_two

LLMChain(prompt=PromptTemplate(input_variables=['recipe'], template='Make a list of cooking ingredients of {recipe} by putting a prefix symbol'), llm=HuggingFaceTextGenInference(inference_server_url='http://eeve-korean-instruct-10-8b-tgi.serving.70-220-152-1.sslip.io', client=<text_generation.client.Client object at 0x7f294c6a3b20>, async_client=<text_generation.client.AsyncClient object at 0x7f294c6a3b80>))

In [12]:
chain_two.invoke({"recipe" : "아보카도를 치즈와 함께 구워먹는다."})

{'recipe': '아보카도를 치즈와 함께 구워먹는다.',
 'text': ' in front of each ingredient.\nIngredient List:\n\n1. 아보카도\n2. 치즈\n3. 소금\n4. 후추\n5. 올리브유\n6. 마늘\n7. 양파\n8. 파프리카\n9. 빵\n10. 후추\n11. 마늘\n12. 파프리카\n13. 올리브유\n14. 소금\n15. 아보카도\n16. 치즈\n17. 소금\n18. 후추\n19. 올리브유\n20. 마늘\n21. 양파\n22. 파프리카\n23. 빵\n24. 후추\n25. 마늘\n26. 파프리카\n27. 올리브유\n28. 소금\n29. 아보카도\n30. 치즈\n31. 소금\n32. 후추\n33. 올리브유\n34. 마늘\n35. 양파\n36. 파프리카\n37. 빵\n38. 후추\n39. 마늘\n40. 파프리카\n41. 올리브유\n42. 소금\n43. 아보카도\n44. 치즈\n45. 소금\n46. 후추\n47. 올리브유\n48. 마늘\n49. 양파\n50. 파프리카\n51. 빵\n52. 후추\n53. 마늘\n54. 파프리카\n55. 올리브유\n56. 소금\n57. 아보카도\n58. 치즈\n59. 소금\n60. 후추\n61. 올리브유\n62. 마늘\n63. 양파\n64. 파프리카\n65. 빵\n66. 후추\n67. 마늘\n68. 파프리카\n69. 올리브유\n70. 소금\n71. 아보카도\n72. 치즈\n73. 소금\n74. 후추\n75. 올리브유\n76. 마늘\n77. 양파\n78. 파프리카\n79. 빵\n80. 후추\n81'}

In [21]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[chain, chain_two],
    verbose=True
)

print (overall_chain.run("파프리카"))



> Entering new SimpleSequentialChain chain...

파프리카 요리는 맛있고 건강한 이탈리아 전통 요리로, 신선한 파프리카를 주재료로 사용합니다. 다음은 맛있고 간편한 파프리카 요리를 만드는 간단한 레시피입니다:

재료:
- 신선한 파프리카 2개
- 올리브유 2스푼
- 다진 마늘 2쪽
- 소금 1작술
- 후춧가루 1작술
- 간 파슬리 2스푼
- 올리브유 1스푼
- 신선한 레몬주스 1스푼

만드는 법:

1. 파프리카를 세로로 반으로 자르고 씨와 껍질을 제거합니다.
2. 큰 팬에 올리브유를 중불에서 달구고 다진 마늘을 넣습니다. 마늘이 갈색이 될 때까지 볶습니다.
3. 준비한 파프리카 조각을 마늘과 함께 팬에 넣고 2-3분 정도 굽니다.
4. 소금과 후춧가루로 간을 하고, 간 파슬리를 뿌립니다.
5. 올리브유와 신선한 레몬주스를 뿌려서 맛을 내줍니다.

이제 맛있고 건강한 파프리카 요리가 완성되었습니다! 이 요리는 쌀이나 빵과 함께 제공하면 맛있고 만족스러운 식사를 하실 수 있습니다.
 in front of your recipe.

> Finished chain.
 in front of your recipe.


In [12]:
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

physics_template = """
You are a very smart physics professor.
You are great at answering questions about physics in a concise and easy to understand manner.
When you don't know the answer to a question you admit that you don't know.
Answer it in Korean, and solve the question step by step.
Start with the answer first, then explain. This is called "deductive reasoning."

Here is a question:
{input}
"""


math_template = """
You are a very good mathematician. You are great at answering math questions.
You are so good because you are able to break down hard problems into their component parts,
answer the component parts, and then put them together to answer the broader question.
Answer it in Korean, and solve the question step by step.
Start with the answer first, then explain. This is called "deductive reasoning."

Here is a question:
{input}
"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")



In [14]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

print(chain.run("블랙홀에 근접하면 시간이 빠르게 흐르는 이유는 뭐지? 한국어로 설명해줘."))



> Entering new MultiPromptChain chain...
physics: {'input': '블랙홀에 가까이 갈수록 시간이 느리게 흐르는 이유는 무엇인가요?'}
> Finished chain.

이유는 다음과 같습니다:

먼저, 중력의 영향을 받는 물체가 시간과 공간의 곡률을 이해하는 것으로 시작해 봅시다. 중력이 강해질수록, 물체는 더 빨리 움직이게 됩니다. 이는 물체의 질량이 중력의 영향을 받아 끌려드는 속도에 비례하기 때문입니다.

이제 블랙홀을 생각해봅시다. 블랙홀은 강한 중력을 가진 천체로, 그 중력은 빛조차도 탈출할 수 없을 정도로 강합니다. 블랙홀에 가까워질수록 물체는 더 빨리 움직이게 되고, 이는 물체가 블랙홀을 향해 움직이는 속도가 블랙홀로부터 멀어질 때보다 더 빠르다는 것을 의미합니다.

이것이 바로 블랙홀에 가까이 갈수록 시간이 느리게 흐르는 이유입니다. 물체가 블랙홀에 가까워질수록, 그 움직임이 더 빨라져 시간을 느리게 만듭니다. 이는 블랙홀의 중력이 물체의 움직임에 미치는 영향으로 인해, 물체가 블랙홀에 가까워질수록 시간이 더 천천히 흐르게 되는 것입니다.

결론적으로, 블랙홀에 가까이 갈수록 시간이 느리게 흐르는 이유는 블랙홀의 강한 중력이 물체의 움직임을 가속화시켜 시간이 더 천천히 흐르게 만들기 때문입니다.


In [15]:
print(
    chain.run(
         "40보다 큰 첫번째 소수이면서, 1을 더하면 3으로 나누어지는 소수는 뭐야?"
        )
    )



> Entering new MultiPromptChain chain...
math: {'input': '40보다 큰 첫번째 소수이면서, 1을 더하면 3으로 나누어지는 소수는 뭐야?'}
> Finished chain.

이 질문에 답하기 위해, 우리는 주어진 조건을 만족하는 소수를 찾아야 합니다.

1단계: 40보다 큰 소수를 찾습니다.
우리는 소수(1과 자기 자신만으로만 나눌 수 있는 수)를 찾고 있습니다. 따라서 2부터 시작해서 40보다 큰 소수인지를 확인해야 합니다.

2부터 40까지 소수 목록을 살펴보면 다음과 같습니다:
2, 3, 5, 7, 11, 13, 17, 19, 23, 29

40보다 큰 소수를 찾기 위해서는 41부터 시작합니다:
41은 40보다 크므로 소수인가요? 아니요, 41은 3과 13으로 나눌 수 있습니다.
43은 40보다 크므로 소수인가요? 예, 43은 소수입니다.

따라서 40보다 큰 첫 번째 소수는 43입니다.

2단계: 1을 더했을 때 3으로 나누어지는지 확인합니다.
소수인 43에 1을 더하면 44가 됩니다.
44를 3으로 나눠봅시다:

44 ÷ 3 = 14 + 2/3

44가 3으로 나누어지므로, 44는 3으로 나누어집니다.

그러므로 40보다 큰 첫 번째 소수로서, 1을 더했을 때 3으로 나누어지는 수는 43입니다.

결론적으로, 40보다 큰 첫 번째 소수로서, 1을 더하면 3으로 나누어지는 수는 43입니다.
